<a href="https://colab.research.google.com/github/zeruel91/DataAnalysis/blob/main/Analysis_UriageOfShop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**제 2장: 대리점 데이터로 데이터를 가공해보자**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

데이터 가공(전처리)는 많은 오류를 포함한 데이터를 가공하는 것이다(입력실수, 데이터누락등등의 오류등을 해결한다).
오류를 포함 데이터를 그냥 분석하게 되면 아예 처리가 안되거나 결과를 심각하게 왜곡한다.d

보통 엑셀로 수작업하여 데이터를 수정하는 일이 빈번한데, 이렇게 하면 오히려 데이터가 더 지저분해지는 경우가 많다(예를 들면 공백은 사람 눈으로 찾기 힘들다).

각기 다른 부서에서 작성한 데이터들은 통합하기가 쉽지 않다.

이 장에서는 실무에서 접하게 되는 다양한 문제해결에 대한 방법들을 살펴본다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
uriage_data = pd.read_csv('uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


**테크닉 11:데이터를 읽어들이자**

In [ ]:
kokyaku_data = pd.read_excel("kokyaku_daicho.xlsx")
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


kokyaku_data는 고객 정보에 관한 데이터이다.

데이터의 입력 오류 또는 표기방법의 차이가 부정합을 일으킬 때 이것을 데이터의 정합성에 문제가 있다고 말한다(쉽게 말해서 결측치가 있다는 뜻이다).

예를 들면, '홍길동'이라고도 하고 '홍 길동'이라고도 하고 '홍 길 동', '홍길 동'이라고도 표기한다.

인간은 데이터의 오류를 보완하면서 데이터를 이해하는데, 시스템은 그렇지 않다(있는 그대로 받아들인다). 그러므로 데이터의 오류를 해소하고 정합성을 보장하는 것은 데이터 분석의 시작이며 아주 중요한 부분이다. 애매모호한 상태로 데이터를 분석하게 되면 결과의 신빙성이나 신뢰성을 확보할 수 없다.

데이터를 파악하는 것부터 시작하자!

**테크닉 12: 데이터를 파악하자!**

In [ ]:
uriage_data["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

상품 A/ 상 품 a/ 상품a 는 각기 다른 상품인 것으로 시스템이 인식한다. 원래 하나의 상품으로 계산해야 하지만, 각기 다른 계산을 하게된다.

In [ ]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

결측치 NaN을 확인할 수 있다. 특히 결측치는 향후 데이터 분석에 큰 영향을 미친다

**테크닉 13: 데이터에 오류가 있는 상태로 집계해보자**

In [ ]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,2,0,1,0,0,0,0,0,...,17,18,15,11,15,17,17,19,18,18,15,21,15,17,21,18,16,7,21,13,12,10,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,20,16,14,13,17,11,13,16,11,15,19,18,19,22,21,21,22,19,21,24,14,11,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,8,26,14,18,12,15,14,20,21,13,11,20,23,16,20,12,23,17,16,21,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,2,0,...,20,17,15,13,11,18,13,14,15,11,20,14,15,19,20,15,15,11,14,13,16,17,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,14,19,17,23,15,16,10,16,12,17,19,18,20,12,22,16,15,16,8,20,16,19,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,...,19,13,18,13,12,18,22,15,15,17,16,14,17,15,16,21,12,17,19,16,14,13,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,17,10,17,19,18,18,23,12,17,11,15,19,26,14,17,22,20,13,26,16,18,12,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0


데이터의 오류가 어느정도 집계에 영향을 미치는지, 데이터의 정합성이 얼마나 중요한지 알아보았다. 위의 데이터는 매출 이력에서 상품별로 월 매출 합계를 집계한 것이다.

(코드 중)1행과 2행에서 날짜를 연,월까지로 변환했다. 

3행에서 세로축에 구입연월, 가로축에 상품의 건수로 집계한다. 4행은 결과를 표시한다.

원래 26개의 상품이 99개로 집계가 되고 있는 것으로 보아, 컬럼이 불필요하게 많다는 것을 확인할 수 있다.

In [ ]:
res = uriage_data.pivot_table(index='purchase_month',columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,1500,1600,2000,0,0,4800,0,1900,0,0,0,0,0,...,6000,6500,7800,7000,10400,13500,16000,16500,19200,19500,21000,27000,20800,25500,36000,28500,28000,12600,41800,27600,24000,20000,0,0,300,0,0,0,0,0,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,0,0,0,0,0,0,0,0,400,0,0,0,0,900,0,0,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7600,5500,6600,7000,12800,8100,11000,14300,13200,16900,23800,25500,30400,30600,32400,32300,40000,37800,39600,43700,31200,20000,2600,0,0,0,0,0,0,1000,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6000,4000,11400,7700,14400,10800,13000,14300,21600,26000,16800,15000,32000,34000,27000,26600,22000,46200,35200,34500,38400,35000,0,0,0,400,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,2000,2200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1500,0,0,0,0,0,7500,0,0,0,0,1200,0,...,7200,8000,7800,9100,7200,14400,13000,12100,15600,14300,22400,18000,22400,28900,30600,28500,28000,16800,19800,20700,38400,32500,0,0,0,0,500,700,0,0,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,0,0,0,100,0,1900,100,0,0,0,0,0,0,0,1100,1300,0,0,0,0,0,2200,0,0,0,0,0,0,600,0,...,5200,9000,9000,14000,12000,13500,8000,15400,12000,20800,23800,24000,28800,15300,32400,20900,26000,33600,8800,36800,31200,45000,0,0,300,0,0,0,0,0,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,0,0,0,0,0,0,0,300,0,0,0,700,0,0,0,0,0,1500,0,0,2100,0,0,0,0,100,0,0,0,800,...,7200,6500,9600,7700,8800,12600,20000,15400,14400,19500,15400,19500,24000,23800,27000,34200,22000,29400,33000,25300,24000,30000,0,0,0,0,0,0,900,0,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1700,0,0,0,0,500,0,0,800,0,0,0,0,1500,0,0,0,2200,0,0,0,0,0,500,0,0,...,6000,4500,9600,11900,12800,13500,17000,9900,20400,13000,18200,25500,38400,18700,25200,38000,34000,23100,52800,32200,38400,27500,0,0,0,0,0,0,900,0,0,0,1500,0,1800,0,0,0,0,0


데이터에 오류가 있는 상태로 집계 및 분석을 하면 전혀 의미없는 결과가 나오게 된다. 이를 통해 데이터 전처리의 중요성을 확인할 수 있다.

**테크닉 14:상품명의 오류를 수정하자**

In [ ]:
print(len(pd.unique(uriage_data['item_name'])))

99


In [ ]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace("  ", "")
uriage_data['item_name'] = uriage_data['item_name'].str.replace(" ", "")
uriage_data.sort_values(by=['item_name'], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [ ]:
print(len(pd.unique(uriage_data['item_name'])))

26


99개의 상품명에 공백을 제거했다. 26개 항목으로 정돈된 것을 확인할 수 있다.

(코드의)1행에서 소문자를 str.upper()대문자로 변환했다.
2행에서 str.replace()로 공백 2개를 제거했다.
3행에서 str.replace()로 공백 1개를 제거했다.
4행에서는 item_name을 오름차순으로 정렬했다.

In [ ]:
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data['item_name'])))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']
26


데이터를 정리한 후 반드시 검증과정을 거쳐서 확인해야한다.

**테크닉 15:금액의 결측치를 수정하자**

In [ ]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

결측치에 따라서 현장에서 조사 후 수정하거나 부서원이 결측치를 채워 넣는 등 여러가지 대응이 가능하다.

In [ ]:
fig_is_null = uriage_data['item_price'].isnull()
for trg in list(uriage_data.loc[fig_is_null, 'item_name'].unique()):
  price = uriage_data.loc[(~fig_is_null) & (uriage_data['item_name'] == trg),
                          'item_price'].max()
  uriage_data['item_price'].loc[(fig_is_null) & (uriage_data['item_name'] == trg)] = price
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


1행에서 item_price 중에 결측치가 있는 곳을 조사한다. 그 결과를 fig_is_null에 저장한다. 

fig_is_null을 이용해서 결측치가 있는 상품명 리스트를 작성한다.

list()는 변수의 값을 리스트 형식으로 변환, loc()는 조건에 일치하는 데이터 추출 메서드다. 

여기서 조건은 '금액에 결측치가 있다'는 것이다.

item_name은 조건과 일치하는 데이터 중에서 어떤 컬럼을 가져올 지 지정한다. 이를 통해 결측치가 존재하는 상품명을 가져온다.

(for문 안의 조건에서) unique()메서드로 중복된 상품명을 제거해준다.

In [ ]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [ ]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
  print(trg+'의 최고가 : ' + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].max())+"의 최저가 : " + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].min(skipna=False)))

상품A의 최고가 : 100.0의 최저가 : 100.0
상품B의 최고가 : 200.0의 최저가 : 200.0
상품C의 최고가 : 300.0의 최저가 : 300.0
상품D의 최고가 : 400.0의 최저가 : 400.0
상품E의 최고가 : 500.0의 최저가 : 500.0
상품F의 최고가 : 600.0의 최저가 : 600.0
상품G의 최고가 : 700.0의 최저가 : 700.0
상품H의 최고가 : 800.0의 최저가 : 800.0
상품I의 최고가 : 900.0의 최저가 : 900.0
상품J의 최고가 : 1000.0의 최저가 : 1000.0
상품K의 최고가 : 1100.0의 최저가 : 1100.0
상품L의 최고가 : 1200.0의 최저가 : 1200.0
상품M의 최고가 : 1300.0의 최저가 : 1300.0
상품N의 최고가 : 1400.0의 최저가 : 1400.0
상품O의 최고가 : 1500.0의 최저가 : 1500.0
상품P의 최고가 : 1600.0의 최저가 : 1600.0
상품Q의 최고가 : 1700.0의 최저가 : 1700.0
상품R의 최고가 : 1800.0의 최저가 : 1800.0
상품S의 최고가 : 1900.0의 최저가 : 1900.0
상품T의 최고가 : 2000.0의 최저가 : 2000.0
상품U의 최고가 : 2100.0의 최저가 : 2100.0
상품V의 최고가 : 2200.0의 최저가 : 2200.0
상품W의 최고가 : 2300.0의 최저가 : 2300.0
상품X의 최고가 : 2400.0의 최저가 : 2400.0
상품Y의 최고가 : 2500.0의 최저가 : 2500.0
상품Z의 최고가 : 2600.0의 최저가 : 2600.0


1행에서 모든 상품에 대해 반복문을 적용하여 상품에 설정된 최대 금액가 최소 금액을 출력해서 검증해본다.
결과로 모든 상품의 최대금액과 최소금액이 일치하는걸로 봐서 성공적으로 금액이 수정되었음을 확인할 수 있다.

min(skipna=False)에서 skipna은 NaN의 무시여부를 설정한다. False로 지정하였기 때문에 NaN이 존재하면 최소값이 NaN으로 표시된다(NaN이 표시되는 경우는 문제가 아직 남아있는 것이다!).

**테크닉 16:고객 이름의 오류를 수정하라**

In [ ]:
kokyaku_data['고객이름'].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [ ]:
uriage_data['customer_name'].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [ ]:
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace(" ","")
kokyaku_data.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04 00:00:00
1,김도윤,E시,42782
2,김지한,A시,2018-01-07 00:00:00
3,김하윤,F시,42872
4,김시온,E시,43127


두 데이터에서 고객이름이 다르게 지정되어있으므로 동일하게 만들어준다.
공백을 제거해서 고객이름의 형식을 통일했다.

In [ ]:
fig_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit()
fig_is_serial.sum()

22

**테크닉 17: 날짜오류를 수정하자**

엑셀 데이터를 취급할 때 주의해야 할 점은 '서식이 다른 데이터가 섞여있다'는 것이다.

In [ ]:
kokyaku_data['등록일']

0      2018-01-04 00:00:00
1                    42782
2      2018-01-07 00:00:00
3                    42872
4                    43127
              ...         
195    2017-06-20 00:00:00
196    2018-06-20 00:00:00
197    2017-04-29 00:00:00
198    2019-04-19 00:00:00
199    2019-04-23 00:00:00
Name: 등록일, Length: 200, dtype: object

In [ ]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[fig_is_serial, '등록일'].astype('float'), unit='D') + pd.to_datetime('1900/01/01')
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

원본 데이터의 등록일에는 날짜데이터와 날짜데이터가 고유번호로 저장된 데이터가 혼영되어있다.

to_timedelta()함수를 이용해서 고유번호 숫자를 진짜 날짜로 변환한다.

In [ ]:
fromString = pd.to_datetime(kokyaku_data.loc[~fig_is_serial, '등록일'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

날짜로 변환된 데이터도 서식을 통일해주었다.

In [ ]:
kokyaku_data['등록일'] = pd.concat([fromSerial, fromString])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


숫자를 날짜로 변경한 데이터fromSerial과 서식을 변경한 fromString을 comcat으로 결합하고 원래 등록일 필드에 대입하여 갱신한다.

In [ ]:
kokyaku_data['등록연월'] = kokyaku_data['등록일'].dt.strftime('%Y%m')
kokyaku_data[['등록일','등록연월']].head()

,등록일,등록연월
0,2018-01-04,201801
1,2017-02-18,201702
2,2018-01-07,201801
3,2017-05-19,201705
4,2018-01-29,201801


등록일에서 연도와 달만 분리해 등록연월 컬럼을 만들어주었다.

In [ ]:
kokyaku_data.groupby(['등록연월']).count()['고객이름']

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64

등록연월로 등록된 고객이름 숫자를 세어보면 위와 같다.

**테크닉 18:고객 이름을 키로 두개의 데이터를 결합(조인)하자**

In [ ]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on = 'customer_name', right_on='고객이름', how='left')
join_data = join_data.drop('customer_name', axis = 1)
join_data

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,201907,김승주,E시,2018-07-15,201807


정합성 없는 데이터들은 서로 다른 열을 지정해서 결합한다. 

left_on으로 좌측 right_on으로 우측데이터를 지정하고 how결합방법을 통해 옆으로 붙인다.

uriage_data를 기준으로 kokyaku_data를 결합한다.


데이터 결합을 통해서 분석에 적합한 데이터 형태가 되었다.

이것을 '데이터 정제'라고 한다.


**테크닉 19:정제한 데이터를 덤프하자**

In [ ]:
dump_data = join_data[['purchase_date','purchase_month','item_name','item_price','고객이름','지역','등록일']]
dump_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15


우리의 목적은 데이터 가공/정제가 아니라 '데이터분석'을 하는 것이다. 

깨끗해지고 정제된 데이터를 파일로 새로 만들어놓고(이를 데이터 덤프라고 한다)
 
 분석을 할 때마다 정제된 파일을 읽어와서 분석하면 매번 정제할 필요가 없다.

 위에서는 컬럼의 위치를 수정해서 dump_data로 저장해주었다. 

 사소한 부분이지만 실제 현장에서도 큰 데이터를 다룰 때 고생하지 않기 위해서 평소에 이런 습관을 길러두는 것이 좋다.

In [ ]:
dump_data.to_csv('dump_data.csv', index=False)

항상 원본은 건드리지 않고 사본을 만들어두는 습관을 들이자!

**테크닉 20:데이터를 집계하자**

In [ ]:
import_data = pd.read_csv('dump_data.csv')
import_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26 00:00:00
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07 00:00:00
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19 00:00:00
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07 00:00:00
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01 00:00:00
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31 00:00:00
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15 00:00:00
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15 00:00:00


In [ ]:
byItem = import_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
byItem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,20,19,19,16,24,16,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,16,11,15,20,19,19,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,14,20,22,13,11,21,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,14,15,11,20,15,15,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,18,13,18,19,18,20,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,15,15,17,16,15,18,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,12,17,11,15,22,26,15,19,23,21,13,28,16,18,12,0


purchase_month를 인덱스 세로축으로 하여 상품별로 집계한다.

In [ ]:
byplace = import_data.pivot_table(index='purchase_month', columns='지역', aggfunc='size', fill_value=0)
byplace

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


purchase_month를 인덱스 세로축으로 하여 지역을 집계한다.